<a href="https://colab.research.google.com/github/gabrielborja/parc_de_montjuic/blob/main/survival_scripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Survival EDA

## Importing libraries

In [ ]:
# Updating libraries version
!pip install matplotlib --upgrade
!pip install plotly --upgrade

In [ ]:
# Importing python libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
from ipywidgets import interact

## Loading Data

In [ ]:
# Loading data from local drive
from google.colab import files
uploaded1 = files.upload()

In [ ]:
# Storing loaded data from csv to a pandas dataframe
import io
df1 = pd.read_csv(io.BytesIO(uploaded1['Scripts_202209051425.csv']), sep='|', engine='python')

In [ ]:
df1.info()

In [ ]:
# Slicing 2 dataframes: numeric and categorical values
df1_a = df1.iloc[:,[0,3,4,5,6,7,8,9,10,11,12,13,14,15,34,35,36,37,38]].copy()
df1_b = df1.iloc[:,[0,3,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38]].copy()

## Exploring Data

In [ ]:
# Listing categories
cat_list_1 = ['MARKET_SEGMENT', 'PRODUCT_TYPE']
cat_list_2 = ['Customer', 'Turnover']

In [ ]:
df1.loc[:,['EVENT_CATEGORY']]

In [ ]:
# Exploring list of columns
@interact(Aggregate_by=cat_list_1, Column_name=df1[1:].columns)
def explore_columns(Aggregate_by, Column_name):
  df = df1.groupby(by=['PERIOD', Aggregate_by, 'EVENT_CATEGORY']).agg(ANT_CHURN = (Column_name, 'sum')).reset_index()
  return df